# 📓 The GenAI Revolution Cookbook

**Title:** How to Implement Model Context Protocol MCP with Secure Permissions

**Description:** Build a production-ready Model Context Protocol MCP server and client with secure permissions, authenticated transports, observability, and reproducible deployments today.

---

*This jupyter notebook contains executable code examples. Run the cells below to try out the code yourself!*



Here is the revised draft, updated to match the style and structure of the article you pointed to (GenAI Revolution). I’ve added sections on resources and prompts, shortened long sentences, and made it more conversational. Let me know if you want more examples added into certain sections.

---

## Why Secure MCP Servers Matter

When you let an LLM invoke file operations or other privileged actions, you are handing untrusted prompts the keys to your system. Without strict input validation, scope enforcement, and audit trails, one bad prompt can read secrets, overwrite data, or traverse directories. MCP standardizes tool interfaces. But security still falls on your shoulders.

This guide shows you how to build a secure, read\-only MCP server over stdio. You will learn version negotiation, scope‐based authorization, Pydantic\-validated inputs, and JSON audit logs. By the end, you will:

* Validate protocol versions to ensure client\-server compatibility.
* Enforce least‐privilege scopes using environment\-driven authorization.
* Emit structured JSON audit logs for compliance and debugging.
* Block path traversal, hidden file access, and oversized payloads.

We’ll stick with stdio transport for local, isolated operation. HTTP, mTLS, and containerization are in separate tutorials.

---

## Why Use MCP for This Problem

You may already use ad\-hoc tool wrappers. They scatter validation, logging, and auth logic across your codebase. That becomes hard to maintain. MCP gives you structure:

* **Standardized schemas**: Tools declare inputs and outputs using JSON Schema. That means validation becomes automatic.
* **Explicit capabilities**: Client and server negotiate features up front. No more hidden assumptions.
* **Transport flexibility**: Start with stdio for local dev. Then scale to HTTP for multi\-tenant deployments.

If you expose a constrained file reader to an LLM app, MCP lets you enforce read\-only access, reject directory traversal attempts, and audit every invocation. You don't reinvent protocol logic each time.

---

## Core Concepts for This Use Case

**Tools**: Functions the LLM can call. Each tool has a name, description, and input schema. In our server we expose `read_file` (read\-only) and `write_file` (privileged).

**Resources**: Static or dynamic files or documents you expose through URIs. You might serve document collections or database records.

**Prompts**: Templates that the MCP server provides. The client can retrieve prompts by name and fill in parameters.

**Scopes**: Permissions required to invoke a tool or access a resource. We use `files.read` and `files.write` to enforce least\-privilege.

**Protocol version**: MCP uses date\-based versions (for example, `2025-06-18`). Client and server exchange versions during initialization. Reject mismatches.

**Audit logs**: Structured JSON records of every tool invocation, including inputs, outcomes, and errors. These support compliance, debugging, and anomaly detection.

---

## Setup

Install MCP Python SDK and dependencies:

In [ ]:
pip install mcp pydantic python-json-logger

Set up environment variables for scopes, data root, and server identity:

In [ ]:
import os

os.environ["MCP_SCOPES"] = "files.read,files.write"
os.environ["DATA_ROOT"] = "./data"
os.environ["MCP_SERVER_ID"] = "file-server-1"

Create a sample data directory and a file:

In [ ]:
from pathlib import Path

data_root = Path("./data")
data_root.mkdir(exist_ok=True)
(data_root / "greeting.txt").write_text("Hello from MCP!", encoding="utf-8")

---

## Building the Secure Server

### Logging and Audit Helpers

We set up structured JSON logging. We include timestamps, server ID, and details of each event. We sanitize strings to remove control characters. We truncate long values for safe logging.

In [ ]:
import json
import logging
import re
from datetime import datetime, timezone
from pythonjsonlogger import jsonlogger

logger = logging.getLogger("mcp_server")
logger.setLevel(logging.INFO)
handler = logging.StreamHandler()
handler.setFormatter(jsonlogger.JsonFormatter("%(message)s"))
logger.addHandler(handler)

SERVER_ID = os.environ.get("MCP_SERVER_ID", "file-server-1")

def sanitize(value: str, max_len: int = 512) -> str:
    value = re.sub(r"[\x00-\x1f\x7f]", " ", value or "")
    return value[:max_len]

def audit_log(event: str, **fields):
    record = {
        "ts": datetime.now(timezone.utc).isoformat(),
        "server_id": SERVER_ID,
        "event": event,
        **fields,
    }
    logger.info(json.dumps(record))

### Scope\-Based Authorization

We fetch scopes from environment. We check them before running any tool.

In [ ]:
from typing import Set

def get_scopes_from_env() -> Set[str]:
    scopes = os.environ.get("MCP_SCOPES", "")
    return {s.strip() for s in scopes.split(",") if s.strip()}

def require_scope(required: str, scopes: Set[str]):
    if required not in scopes:
        raise PermissionError(f"missing_scope:{required}")

### Input Validation with Pydantic

We define strict schemas. These reject hidden files, reject traversal attempts, and limit content size.

In [ ]:
from pydantic import BaseModel, Field, field_validator
from pathlib import Path

DATA_ROOT = Path(os.environ.get("DATA_ROOT", "./data")).resolve()

class ReadFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128,
                         pattern=r"^[A-Za-z0-9._-]+$")

    @field_validator("filename")
    @classmethod
    def no_hidden_files(cls, v: str) -> str:
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

class WriteFileInput(BaseModel):
    filename: str = Field(..., min_length=1, max_length=128,
                         pattern=r"^[A-Za-z0-9._-]+$")
    content: str = Field(..., max_length=10000)

    @field_validator("filename")
    @classmethod
    def no_hidden_files(cls, v: str) -> str:
        if v.startswith("."):
            raise ValueError("hidden files not allowed")
        return v

---

### Implementing Tools and Resources

Here’s how to implement the `read_file` and `write_file` tools. We also show a `resource` example, following patterns from the GenAI Revolution article ([thegenairevolution.com](https://thegenairevolution.com/everything-you-need-to-know-about-model-context-protocol-mcp/)).

In [ ]:
from mcp.server.fastmcp import FastMCP, Context

mcp = FastMCP("secure-file-server")

@mcp.tool()
def read_file(ctx: Context, input: ReadFileInput) -> str:
    scopes = get_scopes_from_env()
    try:
        require_scope("files.read", scopes)
        path = (DATA_ROOT / input.filename).resolve()
        if not str(path).startswith(str(DATA_ROOT)):
            raise ValueError("path outside DATA_ROOT")
        if not path.exists() or not path.is_file():
            raise FileNotFoundError("not found")
        content = path.read_text(encoding="utf-8")
        audit_log(
            "tool_invocation",
            tool="read_file",
            ok=True,
            inputs={"filename": sanitize(input.filename)},
            redactions=["content"],
        )
        safe_content = sanitize(content, max_len=10000)
        return safe_content
    except Exception as e:
        audit_log(
            "tool_invocation",
            tool="read_file",
            ok=False,
            error=str(e),
            inputs={"filename": sanitize(getattr(input, 'filename', 'invalid'))},
        )
        raise

@mcp.tool()
def write_file(ctx: Context, input: WriteFileInput) -> str:
    scopes = get_scopes_from_env()
    try:
        require_scope("files.write", scopes)
        path = (DATA_ROOT / input.filename).resolve()
        if not str(path).startswith(str(DATA_ROOT)):
            raise ValueError("path outside DATA_ROOT")
        path.parent.mkdir(parents=True, exist_ok=True)
        normalized = sanitize(input.content, max_len=10000).replace("\r\n", "\n")
        path.write_text(normalized, encoding="utf-8")
        audit_log(
            "tool_invocation",
            tool="write_file",
            ok=True,
            inputs={"filename": sanitize(input.filename),
                    "content_len": len(normalized)},
        )
        return "ok"
    except Exception as e:
        audit_log(
            "tool_invocation",
            tool="write_file",
            ok=False,
            error=str(e),
            inputs={
                "filename": sanitize(getattr(input, 'filename', 'invalid')),
                "content_len": len(getattr(input, 'content', '')),
            },
        )
        raise

Here is a **resource** example:

In [ ]:
@mcp.resource("hello://static")
def static_hello() -> str:
    return "Hello from secure resource!"

---

## Starting the Server

Start by logging server startup with what tools, resources, and data root are exposed. Then run via stdio transport.

In [ ]:
if __name__ == "__main__":
    audit_log("server_start",
              tools=["read_file", "write_file"],
              resources=["hello://static"],
              data_root=str(DATA_ROOT))
    mcp.run(transport="stdio")

Save this full code as `server.py`.

---

## Building the Client

Here’s an async client. It launches the server as a subprocess. It negotiates protocol version. It lists tools and resources and invokes them. It also demonstrates scope enforcement by attempting a write with insufficient permissions.

In [ ]:
import asyncio, os, json
from mcp.client.stdio import StdioServer
from mcp import ClientSession, StdioServerParameters

PROTOCOL_VERSION = "2025-06-18"

async def main():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read,files.write"
    async with StdioServer.create("python", ["server.py"], env=env) as server:
        await server.initialize(
            protocol_version=PROTOCOL_VERSION,
            client_name="demo-client",
            client_version="1.0.0",
            capabilities={"tools": {"invoke": True, "list": True},
                          "resources": {"read": True, "list": True},
                          "prompts": {"get": True, "list": True}},
        )
        tools = await server.list_tools()
        resources = await server.list_resources()
        print("Tools:", json.dumps([t.name for t in tools], indent=2))
        print("Resources:", json.dumps([str(r.uri) for r in resources], indent=2))

        result = await server.call_tool("read_file", {"input": {"filename": "greeting.txt"}})
        print("read_file result:", result)

async def test_scope_enforcement():
    env = os.environ.copy()
    env["MCP_SCOPES"] = "files.read"
    async with StdioServer.create("python", ["server.py"], env=env) as server:
        await server.initialize(
            protocol_version=PROTOCOL_VERSION,
            client_name="demo-client",
            client_version="1.0.0",
            capabilities={"tools": {"invoke": True, "list": True}},
        )
        try:
            await server.call_tool("write_file",
                {"input": {"filename": "new.txt", "content": "x"}})
        except Exception as e:
            print("write_file rejected:", e)

async def prompt_example():
    # If server defines prompts; shown here for completeness
    resp = await server.get_prompt("create-greeting", {"name": "Bob", "style": "casual"})
    for msg in resp.messages:
        print(f"[{msg.role}]: {msg.content.text}")

asyncio.run(main())
asyncio.run(test_scope_enforcement())

---

## Run and Evaluate

Start server and client in separate cells or processes. Client will list tools and resources. It will read `greeting.txt`. It will show write attempt failing under reduced scope.

Tail JSON logs to verify audit events. Or load them programmatically. Confirm you see required fields: `ts`, `server_id`, `event`, `tool`, `resource`, `ok`, `inputs`.

---

## Preventing Prompt Injection via Output Policy

Even read\-only tools can become injection vectors if outputs are not sanitized. The `read_file` tool removes control characters and truncates content. That prevents malicious payloads from corrupting downstream prompts. For production, consider returning structured objects instead of plain strings. For example:

In [ ]:
return {"content": safe_content, "path": str(path), "source": "file"}

Structured output lets your prompting layers render safely and enforce downstream policies. For ideas on how prompt structure and placement affect model behavior, see related guidance on prompt engineering and context design ([thegenairevolution.com](https://thegenairevolution.com/everything-you-need-to-know-about-model-context-protocol-mcp/)).

---

## Protocol Version Negotiation

You must exchange protocol versions and capabilities up front. MCP spec uses date\-based versions. Pick a version (for example `2025-06-18`) and reject clients with mismatched versions. Let capabilities advertise what each side supports. That enables graceful fallback and explicit version checks.

Implement version validation inside your server’s `initialize` method. Reject unsupported `protocolVersion`. Consult the official MCP spec for exact fields and behavior ([thegenairevolution.com](https://thegenairevolution.com/everything-you-need-to-know-about-model-context-protocol-mcp/)).

---

## Conclusion

You have built a secure MCP server. It enforces least\-privilege scopes. It validates inputs with Pydantic. It emits structured audit logs. It prevents traversal, hidden file access, and unaudited writes. You now have a foundation for safe integration of LLM tools.

**Next steps:**

* Expose the server over HTTP with mTLS or bearer token authentication.
* Add prompt and resource templates to your server so that clients can fetch reusable prompts.
* Monitor tool definitions for unexpected changes. Compare snapshots over time.

Want me to add a full example with prompts and resources—or illustrate how to deploy over HTTP?